In [24]:
using Pkg
Pkg.activate(".")

  Activating new project at `~/woodev/QCAP`


In [26]:
using JuMP, Gurobi

# ----------------------
# 1. 데이터 정의 (샘플 인스턴스)
# ----------------------
N = 4        # 선박 수
T = 24       # 전체 시간 슬롯 (1시간 단위: t=1,...,24)
Q_total = 10 # 전체 크레인 수

a = Dict(1 => 5, 2 => 5, 3 => 5, 4 => 4)


b = Dict(1 => 5, 2 => 3, 3 => 11, 4 => 15)
d = Dict(1 => 8, 2 => 10, 3 => 15, 4 => 22)

available_cranes = Dict(
    1 => [1, 2, 3],
    2 => [6, 7, 8],
    3 => [1, 2, 3],
    4 => [4, 5]
)

qmin = Dict(1 => 0, 2 => 0, 3 => 0, 4 => 0)
qmax = Dict(i => length(available_cranes[i]) for i in keys(available_cranes))

H = 2
C3 = 10    # 계약시간 단가
C4 = 30    # 초과근무 단가
w = Dict(t => (t >= 18 || t <= 4 ? 1.5 : 1.0) for t in 1:T)
M = 50 
M1 = 30

# ----------------------
# 2. 인덱스 집합 전처리
# ----------------------
ships = 1:N
time_slots = 1:T
T_work = Dict(i => b[i]:(d[i]-1) for i in ships)
crane_ships = Dict(m => [i for i in ships if m in available_cranes[i]] for m in 1:Q_total)

# ----------------------
# 3. 모델 생성 및 변수 정의
# ----------------------
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "OutputFlag", 1)

@variable(model, x_c[i in ships, t in time_slots, m in available_cranes[i]], Bin)
@variable(model, x_o[i in ships, t in time_slots, m in available_cranes[i]], Bin)
@variable(model, r_var[i in ships, t in time_slots, m in available_cranes[i]], Bin)
@variable(model, s_var[i in ships, t in time_slots, m in available_cranes[i]], Bin)
@variable(model, e_var[i in ships, t in time_slots, m in available_cranes[i]], Bin)
@variable(model, z[i in ships, t in time_slots, m in available_cranes[i]] >= 0)

# ----------------------
# 4. 제약조건
# ----------------------

# 4.1. 입항 전/출항 후 할당 금지
for i in ships, m in available_cranes[i]
    @constraint(model, [t in time_slots; t ∉ T_work[i]], x_c[i,t,m] == 0)
    @constraint(model, [t in time_slots; t ∉ T_work[i]], x_o[i,t,m] == 0)
end

# 4.2. 총 작업량 충족
@constraint(model, [i in ships],
    sum(r_var[i,t,m] for t in T_work[i], m in available_cranes[i]) == a[i])

# 4.3. 시간 슬롯별 최소/최대 크레인 할당
@constraint(model, [i in ships, t in T_work[i]],
    qmin[i] <= sum(x_c[i,t,m] + x_o[i,t,m] for m in available_cranes[i]) <= qmax[i])

# 4.4. 한 크레인은 한 시간에 단 한 선박에만 할당
@constraint(model, [m in 1:Q_total, t in time_slots],
    sum(x_c[i,t,m] + x_o[i,t,m] for i in crane_ships[m]) <= 1)

@constraint(model, [i in ships, m in available_cranes[i]],
    sum(x_c[i,t,m] for t in T_work[i]) <= H)  # 크레인별 계약 시간 한도

# 4.6. r 변수와의 연결
@constraint(model, [i in ships, t in time_slots, m in available_cranes[i]],
    r_var[i,t,m] == x_c[i,t,m] + x_o[i,t,m])

# 4.7. 연속 블록 제약
@constraint(model, [i in ships, m in available_cranes[i], t in (b[i]+1):(d[i]-1)],
    r_var[i,t,m] - r_var[i,t-1,m] == s_var[i,t,m] - e_var[i,t,m])
@constraint(model, [i in ships, m in available_cranes[i]], s_var[i,b[i],m] == r_var[i,b[i],m])
@constraint(model, [i in ships, m in available_cranes[i]], e_var[i,b[i],m] == 0)
@constraint(model, [i in ships, m in available_cranes[i]], sum(s_var[i,t,m] for t in T_work[i]) == 1)
@constraint(model, [i in ships, m in available_cranes[i]], sum(e_var[i,t,m] for t in T_work[i]) == 1)
@constraint(model, [i in ships, t in T_work[i], m in available_cranes[i]],
    s_var[i,t,m] + e_var[i,t,m] <= 1)
for i in ships
    @constraint(model, sum(t * e_var[i,t,m] for t in T_work[i], m in available_cranes[i]) - 
                       sum(t * s_var[i,t,m] for t in T_work[i], m in available_cranes[i]) >= a[i])
end

# 4.8. 크레인 연속성 제약
for i in ships, t in T_work[i]
    cranes = sort(available_cranes[i])
    for j in 1:length(cranes)-2
        m_val = cranes[j]
        n_val = cranes[j+2]
        m_mid = cranes[j+1]
        @constraint(model, r_var[i,t,m_val] + r_var[i,t,n_val] - 1 <= r_var[i,t,m_mid])
    end
end

# 4.9. z 제한
for i in ships, m in available_cranes[i], t in T_work[i], tp in (t+1):d[i]
    @constraint(model, z[i,t,m] <= (tp - t) + M1 * (1 - e_var[i,tp,m]))
end

# 4.10. z 관련 추가 제약
@constraint(model, [i in ships, t in T_work[i], m in available_cranes[i]], 
    z[i,t,m] <= M * s_var[i,t,m])
@constraint(model, [i in ships, t in T_work[i], m in available_cranes[i]], 
    z[i,t,m] <= r_var[i,t,m])
@constraint(model, [i in ships, t in T_work[i], m in available_cranes[i]], 
    z[i,t,m] <= H)

# ----------------------
# 5. 목적함수
# ----------------------
@objective(model, Min,
    sum(w[t] * (C3 * x_c[i,t,m] + C4 * x_o[i,t,m]) for i in ships, t in time_slots, m in available_cranes[i]))

# ----------------------
# 6. 최적화 및 결과 출력
# ----------------------
optimize!(model)

if termination_status(model) == MOI.OPTIMAL
    println("\nOptimal solution found:")
    println("Objective value = ", objective_value(model))
    for i in ships
        println("\n선박 $i:")
        for m in available_cranes[i]
            println("  크레인 $m:")
            for t in T_work[i]
                println("    t=$t: r = ", round(Int, value(r_var[i,t,m])),
                        ", x^c = ", round(Int, value(x_c[i,t,m])),
                        ", x^o = ", round(Int, value(x_o[i,t,m])),
                        ", s = ", round(Int, value(s_var[i,t,m])),
                        ", e = ", round(Int, value(e_var[i,t,m])),
                        ", z = ", round(Int, value(z[i,t,m])))
            end
        end
    end
else
    println("No optimal solution found. (Status: ", termination_status(model), ")")
end

Set parameter Username
Academic license - for non-commercial use only - expires 2026-02-21
Set parameter OutputFlag to value 1
Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[rosetta2] - Darwin 24.3.0 24D60)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1475 rows, 1605 columns and 3228 nonzeros
Model fingerprint: 0xd274a395
Variable types: 285 continuous, 1320 integer (1320 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [1e+01, 4e+01]
  Bounds range     [2e+00, 3e+00]
  RHS range        [1e+00, 4e+01]
Presolve removed 1410 rows and 1539 columns
Presolve time: 0.02s
Presolved: 65 rows, 66 columns, 246 nonzeros
Variable types: 0 continuous, 66 integer (66 binary)
Found heuristic solution: objective 210.0000000
Found heuristic solution: objective 190.0000000

Root relaxation: cutoff, 0 iterations, 0.00 seconds (0.00 work units)

Ex